In [60]:
!git clone https://github.com/mateoKutnjak/pose_estimation.git

fatal: destination path 'pose_estimation' already exists and is not an empty directory.


In [0]:
import sys
sys.path.append('/content/pose_estimation/models.py')
sys.path.append('/content/pose_estimation/dataset.py')

In [0]:
from google.colab import drive

import os
import pprint

from models import HourglassModel
from dataset import MPII_dataset
import tensorflow as tf
import keras.backend as k
import argparse

In [45]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [46]:
if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
    
    print('TPU devices:')
    pprint.pprint(devices)

('TPU address is', 'grpc://10.78.243.34:8470')
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15541828444616621382),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5460533348251329227),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4694880206961824073),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 9885674210246675583),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17915884063853308500),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5055963834431483901),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 12833562653925877142),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 4125706877236364996),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 123789

In [0]:
IMAGES_DIR = '/content/drive/My Drive/images'
ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/annotations/annotations.json'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8 * 64
STACKS_NUM = 4

In [0]:
mpii_dataset = MPII_dataset(
        images_dir=IMAGES_DIR,
        annots_json_filename=ANNOTATIONS_JSON_FILE,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
        type='train'
    )

In [0]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 1.0

# k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
hg_model = HourglassModel(
    dataset=mpii_dataset,
    batch_size=BATCH_SIZE,
    stacks_num=STACKS_NUM,
)

In [56]:
hg_model.build()

Instructions for updating:
Colocations handled automatically by placer.


In [57]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

hg_model.model = tf.contrib.tpu.keras_to_tpu_model(
    hg_model.model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

INFO:tensorflow:Querying Tensorflow master (grpc://10.78.243.34:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 15541828444616621382)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5460533348251329227)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4694880206961824073)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 9885674210246675583)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17915884063853308500)
INFO:tensorflow:*** Available Device: _DeviceAttributes(

ValueError: ignored

In [0]:
hg_model.train()